# German_Traffic_Sign_Recognition
## About Dataset
### Context
The [German Traffic Sign Benchmark](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign?datasetId=82373)
 is a multi-class, single-image classification challenge held at the International Joint Conference on Neural Networks (IJCNN) 2011. We cordially invite researchers from relevant fields to participate: The competition is designed to allow for participation without special domain knowledge. Our benchmark has the following properties:

* Single-image, multi-class classification problem
* More than 40 classes
* More than 50,000 images in total
* Large, lifelike database

**Acknowledgements** \
INI Benchmark Website


## Imports

In [1]:
import logging
import os

# ignore all the warning and debug information from tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

from sklearn.metrics import classification_report
from libs.nn.conv.lenet import LeNet
from libs.nn.conv.minivggnet import MiniVGGNet
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from pathlib import Path
import os

In [2]:

# Avoid OOM errors by setting GPU Memory Consumption Growth
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus: 
#     tf.config.experimental.set_memory_growth(gpu, True)
# tf.config.list_physical_devices('GPU')

## Prepare Dataset for training

In [3]:
# Look into the data directory
images_dir = './gtsrb-german-traffic-sign'

dataset_dir = f'{images_dir}/Train'

In [4]:
BATCH_SIZE = 32
IMG_WIDTH = IMG_HEIGHT = 64
RANDOM_STATE=123

In [5]:
classes = {0: 'Speed limit (20km/h)',
           1: 'Speed limit (30km/h)',
           2: 'Speed limit (50km/h)',
           3: 'Speed limit (60km/h)',
           4: 'Speed limit (70km/h)',
           5: 'Speed limit (80km/h)',
           6: 'End of speed limit (80km/h)',
           7: 'Speed limit (100km/h)',
           8: 'Speed limit (120km/h)',
           9: 'No passing',
           10: 'No passing veh over 3.5 tons',
           11: 'Right-of-way at intersection',
           12: 'Priority road',
           13: 'Yield',
           14: 'Stop',
           15: 'No vehicles',
           16: 'Veh > 3.5 tons prohibited',
           17: 'No entry',
           18: 'General caution',
           19: 'Dangerous curve left',
           20: 'Dangerous curve right',
           21: 'Double curve',
           22: 'Bumpy road',
           23: 'Slippery road',
           24: 'Road narrows on the right',
           25: 'Road work',
           26: 'Traffic signals',
           27: 'Pedestrians',
           28: 'Children crossing',
           29: 'Bicycles crossing',
           30: 'Beware of ice/snow',
           31: 'Wild animals crossing',
           32: 'End speed + passing limits',
           33: 'Turn right ahead',
           34: 'Turn left ahead',
           35: 'Ahead only',
           36: 'Go straight or right',
           37: 'Go straight or left',
           38: 'Keep right',
           39: 'Keep left',
           40: 'Roundabout mandatory',
           41: 'End of no passing',
           42: 'End no passing veh > 3.5 tons'}

In [6]:
len(classes.keys())

43

In [7]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
  dataset_dir,
  validation_split=0.2,
  subset="both",
  seed=RANDOM_STATE,
  shuffle=True,
  label_mode='categorical' ,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Using 7841 files for validation.


In [8]:
classes  = train_ds.class_names
classes

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '5',
 '6',
 '7',
 '8',
 '9']

In [9]:
for x, y in train_ds.take(1):
    print(x)
    print(y)
    break

tf.Tensor(
[[[[ 95.         96.         96.       ]
   [ 95.36719    95.265625   94.16406  ]
   [ 95.94531    94.109375   91.27344  ]
   ...
   [ 71.94531    75.94531    64.890625 ]
   [ 71.36719    75.36719    63.734375 ]
   [ 71.         75.         63.       ]]

  [[ 96.5625     97.5625     98.734375 ]
   [ 96.06909    96.254395   96.32471  ]
   [ 95.29224    94.194824   92.53076  ]
   ...
   [ 71.59741    75.57605    64.91199  ]
   [ 71.47095    75.223755   63.981567 ]
   [ 71.390625   75.         63.390625 ]]

  [[ 98.9375     99.9375    102.890625 ]
   [ 97.13599    97.757324   99.60889  ]
   [ 94.29956    94.32471    94.441895 ]
   ...
   [ 71.0686     75.01477    64.94446  ]
   [ 71.62866    75.00574    64.3573   ]
   [ 71.984375   75.         63.984375 ]]

  ...

  [[ 97.90625   103.921875   83.90625  ]
   [ 95.726074  102.4646     82.448975 ]
   [ 92.29346   100.170166   80.15454  ]
   ...
   [ 55.976562   57.138916   50.19104  ]
   [ 55.398438   58.2771     51.88025  ]
   [ 

Normalization

In [10]:
normalization_layer = keras.Sequential(
    [
        layers.Rescaling(1. / 255)
    ]
)

labels are already one-hot encoded, now we have to normalize the images, that will be done in side the model (1st Sequential layer)

Augmentation

In [11]:
data_augmentation = keras.Sequential(
    [
        layers.RandomRotation(0.3),
        layers.RandomZoom(0.2),
        layers.RandomFlip("horizontal_and_vertical"),
    ]
)

In [12]:
AUTOTUNE = tf.data.AUTOTUNE


def prepare(ds, shuffle=False, augment=False):
    # Resize and rescale all datasets.
    ds = ds.map(lambda x, y: (normalization_layer(x), y),
                num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Use data augmentation only on the training set.
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=AUTOTUNE)

    # Use buffered prefetching on all datasets.
    return ds.prefetch(buffer_size=AUTOTUNE)

In [13]:
train_aug_ds = prepare(train_ds, shuffle=False, augment=False)
val_ds = prepare(val_ds)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Prepare CNN Model

Setting up the callbacks

In [14]:
# early_stopping_cb = keras.callbacks.EarlyStopping(patience=20,
#                                                      restore_best_weights=True)
                                                     
# SAVED_MODEL_PATH = 'model/ge_traffic_sign_recognition.h5'
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint(SAVED_MODEL_PATH, 
#                                                       monitor="val_loss", 
#                                                       mode="min",
#                                                       save_best_only=True, 
#                                                       verbose=1)

# run_index = 1 # increment every time you train the model
# run_logdir = Path() / "ge_traffic_sign_recog" / f"run_{run_index:03d}"
# tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
# callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]
callbacks = []

In [15]:
model = MiniVGGNet.build(width=IMG_WIDTH, height=IMG_HEIGHT, depth= 3, classes=len(classes))

In [16]:
EPOCHS = 100
lr = 1e-2
optimizer = tf.keras.optimizers.SGD(learning_rate=lr , weight_decay=lr /EPOCHS, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                      

Configure the dataset for performance

In [17]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Train the Model

In [18]:
# tf.keras.backend.clear_session()
tf.random.set_seed(RANDOM_STATE)

In [19]:
history = model.fit(train_ds,
                        validation_data=val_ds,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        callbacks=callbacks)

Epoch 1/100


2023-02-18 20:58:34.575106: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


981/981 [==============================] - 10s 8ms/step - loss: 0.8506 - accuracy: 0.7738 - val_loss: 5.5023 - val_accuracy: 0.0533
Epoch 2/100
981/981 [==============================] - 8s 8ms/step - loss: 0.1105 - accuracy: 0.9672 - val_loss: 18.2835 - val_accuracy: 0.0172
Epoch 3/100
981/981 [==============================] - 8s 8ms/step - loss: 0.0619 - accuracy: 0.9821 - val_loss: 45.9093 - val_accuracy: 0.0170
Epoch 4/100
981/981 [==============================] - 8s 8ms/step - loss: 0.0351 - accuracy: 0.9891 - val_loss: 39.3728 - val_accuracy: 0.0292
Epoch 5/100
981/981 [==============================] - 8s 8ms/step - loss: 0.0192 - accuracy: 0.9942 - val_loss: 71.9695 - val_accuracy: 0.0564
Epoch 6/100
981/981 [==============================] - 8s 8ms/step - loss: 0.0159 - accuracy: 0.9953 - val_loss: 20.4278 - val_accuracy: 0.0551
Epoch 7/100
981/981 [==============================] - 8s 8ms/step - loss: 0.0095 - accuracy: 0.9970 - val_loss: 35.9991 - val_accuracy: 0.0101
Epoc

KeyboardInterrupt: 

## Evaluate the Model

In [ ]:
history.history.keys()

In [ ]:
plt.figure()
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy per Epoch")

plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

In [ ]:
loss, accuracy = model.evaluate(val_ds)
loss, accuracy

In [ ]:
predicted = model.predict(val_ds)
for pred in predicted[:3]:
    print(np.argmax(pred, axis=-1))

In [ ]:
def get_actual_predicted_labels(dataset): 
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  actual =np.argmax(actual, axis=1) # because one hot encoded 
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [ ]:
actual, predicted = get_actual_predicted_labels(val_ds)
actual[:10], predicted[:10].numpy()

In [ ]:
print(classification_report(actual, predicted,
                            target_names=classes.values()))